In [ ]:
pwd

In [1]:
# 라이브러리 import
import os
import pandas as pd
import numpy as np
import math 

# 셀레늄
from selenium.webdriver.common.alert import Alert
from selenium import webdriver  # 라이브러리(모듈) 가져오라
# pip install chromedriver_autoinstaller
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains as AC

# tqdm : for문 진행상황 체크
from tqdm import tqdm, tqdm_notebook
from tqdm.notebook import tqdm

# 정규표현식(regular expression) : 문자(알파벳,한글), 숫자, 특수기호 정제 및 추출
import re
from time import sleep
import time

# 워닝 무시
import warnings
warnings.filterwarnings('ignore')

In [2]:
# 영화배우 이름 검색
name = '배우 이정재'

# 크롬 옵션
options = webdriver.ChromeOptions()

# 크롬 윈도우 사이즈 조절
options.add_argument("--window-size=800,1200") # window-size -> 기본 : 1920,1080

import chromedriver_autoinstaller
chrome_path = chromedriver_autoinstaller.install()
driver = webdriver.Chrome(chrome_path, options=options)
time.sleep(3)
driver.get("https://www.naver.com")
# -> 네이버 크롬 창이 뜬다.

In [3]:
# 네이버 검색어 입력 후 검색
element = driver.find_element("name", "query") # by_name
element.clear()  # 혹시 검색창에 존재하는 텍스트 제거

element.send_keys(name)  # 검색창에 검색어 전달
element.submit()  # 검색 클릭
time.sleep(1)

# 프로필 클릭
driver.find_element("link text", "프로필").click()  # by_link_text
time.sleep(1)

# 1. 배우 수상내역 수집하기

In [4]:
# 수상내역 더보기 클릭
try:
    driver.find_element("css selector", ".area_button_arrow").click()  # by_css_selector
    time.sleep(1)
except:
    pass

In [5]:
# 수상내역 크롤링
prize_list = []

In [6]:
prize = driver.find_element("css selector", ".cm_content_area._cm_content_area_prize").text
 # by_css_selector
prize

'수상내역\n2022\n제1회 청룡시리즈어워즈 드라마부문 남우주연상 (오징어 게임)\n2022\n제27회 크리틱스 초이스 시상식 TV드라마시리즈부문 남우주연상 (오징어 게임)\n2022\n제37회 인디펜던트 스피릿 어워드 TV부문 남우주연상 (오징어 게임)\n2022\n제28회 미국배우조합상 TV드라마시리즈부문 남우주연상 (오징어 게임)\n2021\n아시아 아티스트 어워즈 대상 올해의 배우\n2021\n아시아 아티스트 어워즈 배우부문 페뷸러스상\n2021\n아시아 아티스트 어워즈 배우부문 핫트렌드상\n2020\n아시아 아티스트 어워즈 대상 올해의 배우 영화부문\n2016\n스타일아이콘어워즈 본상\n2015\n제3회 마리끌레르 아시아 스타 어워즈 올해의 배우상\n2015\n제24회 부일영화상 남우주연상\n2015\n제51회 백상예술대상 InStyle상\n이전\n현재\n1\n전체\n3 다음\n펴고 닫기'

In [7]:
# 년도만 세기
import re
p = re.compile('[0-9]{4,4}')  # 정규표현식 - 숫자 4자리만 추출하라
m = p.findall(prize)
m

['2022',
 '2022',
 '2022',
 '2022',
 '2021',
 '2021',
 '2021',
 '2020',
 '2016',
 '2015',
 '2015',
 '2015']

In [8]:
prize_list = prize_list + m
prize_list

['2022',
 '2022',
 '2022',
 '2022',
 '2021',
 '2021',
 '2021',
 '2020',
 '2016',
 '2015',
 '2015',
 '2015']

In [9]:
# 수상내역 다음페이지 클릭
if driver.find_element("css selector", ".pg_next.on"):   # by_css_selector
    next = driver.find_element("css selector", ".pg_next.on").click() # by_css_selector
    time.sleep(2)

# 배우 수상내역 페이지 for문으로 묶기

In [10]:
# 영화배우 이름 검색
name = '배우 이정재'

# 크롬 옵션
options = webdriver.ChromeOptions()  

# 크롬 윈도우 사이즈 조절
options.add_argument("--window-size=800, 1200") # window-size -> 기본 : 1920,1080

import chromedriver_autoinstaller
chrome_path = chromedriver_autoinstaller.install()
driver = webdriver.Chrome(chrome_path, options=options)
time.sleep(3)
driver.get("https://www.naver.com")

# 네이버 검색어 입력 후 검색
element = driver.find_element("name", "query")
element.clear()  # 혹시 검색창에 존재하는 텍스트 제거

element.send_keys(name)  # 검색창에 검색어 전달
element.submit()  # 검색 클릭
time.sleep(1)

# 프로필 클릭
driver.find_element("link text", "프로필").click()
time.sleep(1)

# 수상내역 더보기 클릭
try:
    driver.find_element("css selector", ".cm_content_area._cm_content_area_prize .area_button_arrow").click()
    time.sleep(1)
except:
    pass

# 수상내역 갯수 세기
prize_list = []

for i in range(0, 100):  # 수상내역 페이지 최대 100페이지까지 수집할 수 있게 설정
    try:
        # 수상내역 크롤링
        prize = driver.find_element("css selector", ".cm_content_area._cm_content_area_prize").text
        # 년도만 세기
        p = re.compile('[0-9]{4,4}')   # 정규표현식으로 "0-9숫자 4자리" 수집
        m = p.findall(prize)           # prize 중에서 모두 찾아라(findall 함수)
        prize_list = prize_list + m    # 페이지 별 수상내역 리스트 전부 합치기

        # 수상내역 다음페이지 클릭
        if driver.find_element("css selector", '.pg_next.on'):
            next = driver.find_element("css selector", '.pg_next.on').click()
            time.sleep(2)
    except:  
        break
    
print('수상 수: ', len(prize_list))
prize_list

수상 수:  29


['2022',
 '2022',
 '2022',
 '2022',
 '2021',
 '2021',
 '2021',
 '2020',
 '2016',
 '2015',
 '2015',
 '2015',
 '2014',
 '2013',
 '2013',
 '2013',
 '2011',
 '2010',
 '2008',
 '2006',
 '2001',
 '1999',
 '1999',
 '1999',
 '1997',
 '1995',
 '1995',
 '1995',
 '1995']

# 2. 배우 필모그래피 수집하기

In [18]:
# 필모그래피 클릭
driver.find_element("link text", "필모그래피").click() # by_link_text
time.sleep(1)

In [19]:
# 인기순 클릭
driver.find_element("link text", "인기순").click()  # by_link_text
time.sleep(1)

In [20]:
driver.find_element('css selector', '.this_text_number').text  # by_css_selector

'36'

In [21]:
film_num = int(driver.find_element('css selector', '.this_text_number').text) # by_css_selector
print('필모 수: ', film_num)

필모 수:  36


In [22]:
# 페이지 클릭횟수 계산
page_click = math.ceil(film_num/4) - 1
print('페이지 클릭 횟수: ', page_click)

페이지 클릭 횟수:  8


In [23]:
# 영화 데이터 수집

score_list = []
cast_list = []
audiance_list = []
main_sub_list = []

In [24]:
title_list = []

# 제목
titles = driver.find_elements('css selector', '.this_text') # by_css_selector
temp_list = []
for title in titles:
    temp_list.append(title.text)
    # ['도둑들', '상영중', '암살', '상영중', '']
    # 리스트 컴프리헨션 : for문 + 리스트
    temp_list = [x for x in temp_list if x !='' and x != '상영중']
#     print(temp_list)    
title_list = title_list + temp_list 
title_list

['신과함께-죄와 벌', '도둑들', '암살', '신과함께-인과 연']

In [25]:
# 배역
cast_list = []
cast_temp = []
for t in range(1, 5):
    try:
        cast = driver.find_element('css selector''#mflick > div:nth-child({}) > div > div > div:nth-child({}) > div.data_area > div > span'.format(i+1, t)).text[:2]
        cast_temp.append(cast)
    except:
        break
cast_list = cast_list + cast_temp        
cast_list
#     print(cast_temp)


[]

In [27]:
i = 0

# 제목 수집 해보기

# 제목 전체 리스트
title_list = []

# 제목 페이지별 리스트
temp_list = []

titles = driver.find_elements('css selector', '.this_text')
title = titles[0]
print(title.text)

신과함께-죄와 벌


## 배우 필모그래피 for문으로 수집하기

In [ ]:
# 배우 리스트에서 이름 불러와서 name 변수에 저장하고 필모그래피 크롤링하기
# pd.read_csv("actor_list.csv")

In [67]:
# 영화배우 이름 검색
name = '배우 송강호'

# 크롬 옵션
options = webdriver.ChromeOptions()

# 크롬 윈도우 사이즈 조절
options.add_argument("--window-size=800,1200") # window-size -> 기본 : 1920,1080

import chromedriver_autoinstaller
chrome_path = chromedriver_autoinstaller.install()
driver = webdriver.Chrome(chrome_path, options=options)
time.sleep(3)
driver.get("https://www.naver.com")

# 네이버 검색어 입력 후 검색
element = driver.find_element("name", "query")
element.clear()  # 혹시 검색창에 존재하는 텍스트 제거

element.send_keys(name)  # 검색창에 검색어 전달
element.submit()  # 검색 클릭
time.sleep(1)

# ctrl shift -  : 셀 나누기
# shift m :  셀 합치기

In [68]:
# 필모그래피 클릭
driver.find_element("link text", "필모그래피").click()
time.sleep(1)

# 인기순 클릭
driver.find_element("link text", "인기순").click()
time.sleep(1)

film_num = int(driver.find_element('css selector', '.this_text_number').text)
print('필모 수: ', film_num)

# 페이지 클릭횟수 계산
page_click = math.ceil(film_num/4) - 1
print('페이지 클릭 횟수: ', page_click)

필모 수:  38
페이지 클릭 횟수:  9


In [69]:
# 영화 데이터 수집
title_list = []
score_list = []
cast_list = []
audiance_list = []
main_sub_list = []

for i in range(page_click):    

    # 제목
    titles = driver.find_elements('css selector', '.this_text')
    temp_list = []
    for title in titles:
        temp_list.append(title.text)
        temp_list = [x for x in temp_list if x !='' and x != '상영중']
#     print(temp_list)    
    title_list = title_list + temp_list    
    
    # 배역
    cast_temp = []
    for t in range(1, 5):
        try:
            cast = driver.find_element('css selector', '#mflick > div:nth-child({}) > div > div > div:nth-child({}) > div.data_area > div > span'.format(i+1, t)).text[:2]
            cast_temp.append(cast)
        except:
            break
    cast_list = cast_list + cast_temp        
#     print(cast_temp)
    
    # 관객수
    score_temp = []
    for j in range(1, 5):
        try:
            score = driver.find_element('css selector', '#mflick > div:nth-child({}) > div > div > div:nth-child({}) > div.data_area > div > div.info > dl:nth-child(2)'.format(i+1, j)).text.split('\n')[1]
            score = score.replace('만','0000').replace(',','')
            if '.' in score:
                score = int(score.replace('.', ''))
                score = round(score*0.1)
            else:
                score = round(int(score))
            
            score_temp.append(score)
        except:
            break
    score_list = score_list + score_temp
#     print(score_temp, '\n')
    if len(score_temp) < 4:
        break
        
    # 필모 다음페이지 클릭
    driver.find_element('css selector', '.pg_next.on').click()
    time.sleep(2)

title_list = title_list[:len(score_list)]
cast_list = cast_list[:len(score_list)]

print('\n', len(title_list))
print(title_list)
print(cast_list)
print(score_list)


 23
['택시운전사', '변호인', '괴물', '기생충', '설국열차', '관상', '밀정', '좋은 놈, 나쁜 놈, 이상한 놈', '사도', '의형제', '친절한 금자씨', '박쥐', '마약왕', '하울링', '밀양', '마다가스카', '브로커', '우아한 세계', '나랏말싸미', '남극일기', '푸른소금', '효자동 이발사', '작은 연못']
['주연', '주연', '주연', '주연', '주연', '주연', '주연', '주연', '주연', '주연', '우정', '주연', '주연', '주연', '주연', '성우', '주연', '주연', '주연', '주연', '주연', '주연', '단역']
[12180000, 11370000, 10910000, 10310000, 9350000, 9130000, 7500000, 6680000, 6240000, 5410000, 3120000, 2210000, 1860000, 1610000, 1600000, 1390000, 1260000, 960000, 950000, 870000, 770000, 740000, 46000]


In [70]:
# 
df = pd.DataFrame(list(zip(title_list, cast_list, score_list)), columns = ['제목','배역', 'score'])
print('관객수 데이터가 있는 영화들만: ')
print(df)

관객수 데이터가 있는 영화들만: 
                   제목  배역     score
0               택시운전사  주연  12180000
1                 변호인  주연  11370000
2                  괴물  주연  10910000
3                 기생충  주연  10310000
4                설국열차  주연   9350000
5                  관상  주연   9130000
6                  밀정  주연   7500000
7   좋은 놈, 나쁜 놈, 이상한 놈  주연   6680000
8                  사도  주연   6240000
9                 의형제  주연   5410000
10            친절한 금자씨  우정   3120000
11                 박쥐  주연   2210000
12                마약왕  주연   1860000
13                하울링  주연   1610000
14                 밀양  주연   1600000
15              마다가스카  성우   1390000
16                브로커  주연   1260000
17             우아한 세계  주연    960000
18              나랏말싸미  주연    950000
19               남극일기  주연    870000
20               푸른소금  주연    770000
21            효자동 이발사  주연    740000
22              작은 연못  단역     46000


In [71]:
# 주연 스코어 합
cast_main = df[(df['배역']=='주연')]
main_score = cast_main['score'].sum()

# 조연 스코어 합
cast_sub = df[(df['배역']=='조연')]
sub_score = cast_sub['score'].sum()

# 단역 스코어 합
cast_other = df[(df['배역']!='주연') & (df['배역']!='조연')]
cast_score = cast_other['score'].sum()

# 배우 흥행지수
actor_score_index = round((main_score*0.5 + sub_score*0.4 + cast_score*0.1) / len(df))
print('{} 배우 흥행지수: '.format(name), actor_score_index)

배우 송강호 배우 흥행지수:  2235243


In [72]:
cast_main = df[(df['배역']=='주연')]
cast_main

,제목,배역,score
0,택시운전사,주연,12180000
1,변호인,주연,11370000
2,괴물,주연,10910000
3,기생충,주연,10310000
4,설국열차,주연,9350000
5,관상,주연,9130000
6,밀정,주연,7500000
7,"좋은 놈, 나쁜 놈, 이상한 놈",주연,6680000
8,사도,주연,6240000
9,의형제,주연,5410000
